Importacoes importantes para a implementacao

In [21]:
import pandas as pd

definicao dos limites e valores ideais.

In [22]:
limits = {
    'Dissolved oxygen': 10.0,
    'pH': 8.5,
    'Conductivity': 1000.0,  # µS/cm
    'Nitrate': 45.0,  # mg/L
    'Biological oxygen demand': 5.0,  # mg/L
    'Fecal coliform': 100.0,  # /100 mL
    'Total coliform': 1000.0  # /100 mL
}

ideal_referencial_value = {
    'Dissolved oxygen': 14.6,  # DO ideal em mg/L
    'pH': 7.0,
    'Conductivity': 0.0,
    'Nitrate': 0.0,
    'Biological oxygen demand': 0.0,
    'Fecal coliform': 0.0,
    'Total coliform': 0.0
}


Carregamento do CSV com os dados.

In [23]:
df = pd.read_csv('/Users/arthurhenrique/Documentos/pessoal/faculdade/documentos/tcc/repository/machine-learning-water-quality/notebook/base_consolidada/amostra_por_classificador.csv')

# for col in ["17_OD_mgl_02", "18_PH", "50_Condutivida_de_Eletrica_us_cm_a_20c", "13_Nitratos_mgl_n)", "46_Coliformes_Fecais_nmp_100ml", "47_Coliformes_Totais_nmp_100ml","7_DBO_mgl_02)"]:
#      df[col] = pd.to_numeric(df[col], errors='coerce')

# df.rename(columns={
#      '17_OD_mgl_02': 'Dissolved oxygen',
#      '18_PH': 'pH',
#      '50_Condutivida_de_Eletrica_us_cm_a_20c': 'Conductivity',
#      '13_Nitratos_mgl_n)': 'Nitrate',
#      '46_Coliformes_Fecais_nmp_100ml': 'Fecal coliform',
#      '47_Coliformes_Totais_nmp_100ml': 'Total coliform',
#      '7_DBO_mgl_02)': 'Biological oxygen demand'
#  }, inplace=True)

# for col in ['Dissolved oxygen', 'pH', 'Conductivity', 'Nitrate', 'Fecal coliform', 'Total coliform', 'Biological oxygen demand']:
#     df[col] = pd.to_numeric(df[col], errors='coerce')

df.drop('WQI', axis=1)


,Dissolved oxygen,pH,Conductivity,Nitrate,Fecal coliform,Total coliform,Biological oxygen demand
0,8.35,5.71,2.26,0.1000,10.0,5172.0,1.0
1,0.00,0.00,0.00,0.0000,0.0,0.0,0.0
2,8.65,5.51,6.65,0.0300,1.0,109.2,1.0
3,6.48,5.30,3.26,0.0200,10.0,960.0,1.0
4,7.30,5.43,4.00,0.0298,20.0,496.0,1.0
...,...,...,...,...,...,...,...
703,7.40,7.10,47.50,0.3000,2200.0,30000.0,2.0
704,8.00,6.64,130.10,0.3800,7000.0,14000.0,2.0
705,4.10,6.25,173.30,0.0700,5000.0,5000.0,5.0
706,7.10,7.20,249.50,0.7500,50.0,300.0,6.0


calculo do wqi

In [24]:

def calcular_qi(Vi, VIdeal, Si):
    return 100 * (Vi - VIdeal) / (Si - VIdeal)

def calcular_wi(Si, K):
    return K / Si

def classify_wqi(wqi):
    if wqi <= 25:
        return 1
    elif wqi <= 50:
        return 2
    elif wqi <= 75:
        return 3
    else:
        return 4


def calcular_wqi_dataframe(df, ideal_referencial_value, limits, type):
    # Calcular a constante K
    K = 1 / sum(limits.values())
    
    wqi_list = []
    
    for idx, row in df.iterrows():
        somatorio_qi_wi = 0
        somatorio_wi = 0
        
        for parametro in limits.keys():
            # Certifique-se de que os valores são convertidos para float
            Vi = float(row[parametro])
            VIdeal = float(ideal_referencial_value[parametro])
            Si = float(limits[parametro])
            
            qi = calcular_qi(Vi, VIdeal, Si)
            wi = calcular_wi(Si, K)
            
            somatorio_qi_wi += qi * wi
            somatorio_wi += wi
        
        # Evitar divisão por zero
        if somatorio_wi != 0:
            wqi = somatorio_qi_wi / somatorio_wi
        else:
            wqi = None  # Ou algum valor padrão, se preferir
        
        if type:
            wqi_list.append(wqi)
        else:
            wqi_list.append(classify_wqi(wqi))
    
    return wqi_list

# Aplicar a função ao DataFrame e adicionar a coluna WQI
df['WQI'] = calcular_wqi_dataframe(df, ideal_referencial_value, limits, True)

# Salvar o DataFrame resultante
df.to_csv('/Users/arthurhenrique/Documentos/pessoal/faculdade/documentos/tcc/repository/machine-learning-water-quality/notebook/base_consolidada/amostra_por_classificador_wqi.csv', index=False)



Metricas dos dados

In [25]:
df['WQI'].value_counts()

WQI
-51.260024     9
 55.831117     2
 207.033843    2
 64.690568     2
 51.152870     2
              ..
 47.112877     1
 26.177414     1
 37.154108     1
 27.049571     1
 96.696887     1
Name: count, Length: 691, dtype: int64